In [5]:
%pip install h2o sklearn

Defaulting to user installation because normal site-packages is not writeable
Processing ./.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19097831/tmpfjwomu57
  JVM stdout: /state/partition1/job-19097831/tmpfjwomu57/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19097831/tmpfjwomu57/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,"21 days, 22 hours and 35 minutes"
H2O_cluster_name:,H2O_from_python_mk7516_31f54b
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.543 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup(225)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(6144, 9)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |██
14:08:02.167: XGBoost_1_AutoML_1_20220505_140752 [XGBoost def_2] failed: java.lang.RuntimeException: Error while training XGBoost model

████████
14:10:08.843: GLM_1_AutoML_1_20220505_140752 [GLM def_1] failed: java.lang.UnsupportedOperationException: error cannot be computed: too many classes

█Job request failed Unexpected HTTP error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), will retry after 3s.
█████████
14:20:51.699: GBM_1_AutoML_1_20220505_140752 [GBM def_5] failed: java.lang.OutOfMemoryError: GC overhead limit exceeded


14:20:53.859: XGBoost_2_AutoML_1_20220505_140752 [XGBoost def_1] failed: java.lang.RuntimeException: Error while training XGBoost model

 (failed)


OSError: Job with key $03017f00000132d4ffffffff$_a4d5d502e307643217813832e5ce4837 failed with an exception: java.lang.OutOfMemoryError: GC overhead limit exceeded
stacktrace: 
java.lang.OutOfMemoryError: GC overhead limit exceeded


In [ ]:

lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [9]:
h2o.cluster().shutdown()

H2O session _sid_a08e closed.
